<a href="https://colab.research.google.com/github/diogo-costa-silva/sql-murder-mystery/blob/main/murder_mystery_from_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# getting the database from my github
!wget https://github.com/diogo-costa-silva/assets/raw/main/databases/sql-murder-mystery.db -O sql-murder-mystery.db

--2024-02-21 13:26:26--  https://github.com/diogo-costa-silva/assets/raw/main/databases/sql-murder-mystery.db
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/diogo-costa-silva/assets/main/databases/sql-murder-mystery.db [following]
--2024-02-21 13:26:27--  https://raw.githubusercontent.com/diogo-costa-silva/assets/main/databases/sql-murder-mystery.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3661824 (3.5M) [application/octet-stream]
Saving to: ‘sql-murder-mystery.db’

sql-murder-mystery. 100%[===================>]   3.49M  --.-KB/s    in 0.09s   

2024-02-21 13:26:28 (38.9 MB/s) - ‘sql-mur

In [1]:
# Load the SQL extension
%load_ext sql

In [3]:
# Create a SQLite database
%sql sqlite:///sql-murder-mystery.db

## 1.Initial Clues:

- murder that occurred sometime on Jan 15, 2018 (20180115)

- and that it took place in SQL City

In [4]:
%%sql
SELECT *
FROM crime_scene_report
WHERE type="murder"
AND city="SQL City"
-- AND date="Jan.15, 2018"
AND date=20180115

 * sqlite:///sql-murder-mystery.db
Done.


date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


## 2. Security footage shows 2 witnesses

- 1st - lives at the last house on "Northwestern Dr"

- 2nd - Annabel, lives somewhere on "Franklin Ave"

In [5]:
%%sql
SELECT *
FROM person
WHERE address_street_name="Northwestern Dr"
ORDER BY address_number DESC
LIMIT 1

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


## 3. 1st witness is Morty Schapiro with id 14887

In [6]:
%%sql

SELECT *
FROM person
WHERE address_street_name="Franklin Ave"
AND name LIKE "%Annabel%"
LIMIT 10

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
16371,Annabel Miller,490173,103,Franklin Ave,318771143


## 4. 2nd witness is Annabel Miller with id 16371


In [7]:
%%sql

SELECT *
FROM person
WHERE id IN (14887, 16371)
LIMIT 10

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [8]:
%%sql

SELECT *
FROM interview
WHERE person_id IN (14887, 16371)
LIMIT 10

 * sqlite:///sql-murder-mystery.db
Done.


person_id,transcript
14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


## 5. Found some transcripts:

- Morty: "Get Fit Now Gym" bag started with "48Z" - gold member
- car with a plate that included "H42W"
- Annabel: recognized the killer from her gym last week on January the 9th

In [9]:
%%sql

SELECT *
FROM drivers_license
WHERE plate_number LIKE "%H42W%"
LIMIT 10

 * sqlite:///sql-murder-mystery.db
Done.


id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [10]:
%%sql

SELECT *
FROM drivers_license AS dl
INNER JOIN person AS p on dl.id = p.license_id
WHERE plate_number LIKE "%H42W%"
LIMIT 10

 * sqlite:///sql-murder-mystery.db
Done.


id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,id_1,name,license_id,address_number,address_street_name,ssn
664760,21,71,black,black,male,4H42WR,Nissan,Altima,51739,Tushar Chandra,664760,312,Phi St,137882671
423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius,78193,Maxine Whitely,183779,110,Fisk Rd,137882671


In [11]:
%%sql

SELECT p.*, gf.*
FROM drivers_license AS dl
INNER JOIN person AS p on dl.id = p.license_id
INNER JOIN get_fit_now_member as gf on p.id = gf.person_id
WHERE plate_number LIKE "%H42W%"
LIMIT 10

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn,id_1,person_id,name_1,membership_start_date,membership_status
67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,48Z55,67318,Jeremy Bowers,20160101,gold


In [12]:
%%sql

SELECT p.*, gf.*
FROM drivers_license AS dl
INNER JOIN person AS p on dl.id = p.license_id
INNER JOIN get_fit_now_member as gf on p.id = gf.person_id
WHERE plate_number LIKE "%H42W%"
AND membership_status = "gold"
LIMIT 10

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn,id_1,person_id,name_1,membership_start_date,membership_status
67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,48Z55,67318,Jeremy Bowers,20160101,gold


In [13]:
%%sql

SELECT p.*, ci.*
FROM drivers_license AS dl
INNER JOIN person AS p on dl.id = p.license_id
INNER JOIN get_fit_now_member as gf on p.id = gf.person_id
INNER JOIN get_fit_now_check_in as ci on gf.id=ci.membership_id
WHERE plate_number LIKE "%H42W%"
AND membership_status = "gold"
LIMIT 10

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn,membership_id,check_in_date,check_in_time,check_out_time
67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,48Z55,20180109,1530,1700


In [14]:
%%sql

SELECT p.*, ci.*
FROM drivers_license AS dl
INNER JOIN person AS p on dl.id = p.license_id
INNER JOIN get_fit_now_member as gf on p.id = gf.person_id
INNER JOIN get_fit_now_check_in as ci on gf.id=ci.membership_id
WHERE plate_number LIKE "%H42W%"
AND membership_status = "gold"
AND check_in_date = 20180109
LIMIT 10

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn,membership_id,check_in_date,check_in_time,check_out_time
67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,48Z55,20180109,1530,1700


## 6. Let's see if the killer is Jeremy Bowers

`INSERT INTO solution VALUES (1, 'Jeremy Bowers');`

`SELECT value FROM solution;`


## 7. Yes it is! But there is a Mastermind behind all this!

In [15]:
%%sql

SELECT *
FROM interview
WHERE person_id=67318

 * sqlite:///sql-murder-mystery.db
Done.


person_id,transcript
67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


## 8. The transcript of Jeremy said the following:
- hired by a woman with a lot of money
-  5'5" (65") or 5'7" (67"), red hair, drives a Tesla Model S
-  attended the SQL Symphony Concert 3 times in December 2017

In [16]:
%%sql

SELECT *
FROM drivers_license
WHERE hair_color = "red"
AND height >= 65
AND height <= 67
AND car_make LIKE "%tesla%"
AND car_model LIKE "Model S"
AND gender = "female"
LIMIT 100

 * sqlite:///sql-murder-mystery.db
Done.


id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
202298,68,66,green,red,female,500123,Tesla,Model S
291182,65,66,blue,red,female,08CM64,Tesla,Model S
918773,48,65,black,red,female,917UU3,Tesla,Model S


## 9. There are still 3 people, so let's check the concert event


In [17]:
%%sql

SELECT p.*
FROM drivers_license as dl
INNER JOIN person as p on dl.id=p.license_id
WHERE hair_color = "red"
AND height >= 65
AND height <= 67
AND car_make LIKE "%tesla%"
AND car_model LIKE "Model S"
AND gender = "female"
LIMIT 100

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn
78881,Red Korb,918773,107,Camerata Dr,961388910
90700,Regina George,291182,332,Maple Ave,337169072
99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [18]:
%%sql

SELECT p.*, fb.*
FROM drivers_license as dl
INNER JOIN person as p on dl.id=p.license_id
INNER JOIN facebook_event_checkin as fb on fb.person_id=p.id
WHERE hair_color = "red"
AND height >= 65
AND height <= 67
AND car_make LIKE "%tesla%"
AND car_model LIKE "Model S"
AND gender = "female"
LIMIT 100

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn,person_id,event_id,event_name,date
99716,Miranda Priestly,202298,1883,Golden Ave,987756388,99716,1143,SQL Symphony Concert,20171206
99716,Miranda Priestly,202298,1883,Golden Ave,987756388,99716,1143,SQL Symphony Concert,20171212
99716,Miranda Priestly,202298,1883,Golden Ave,987756388,99716,1143,SQL Symphony Concert,20171229


In [19]:
%%sql

SELECT person_id,
COUNT (*) as visits
FROM facebook_event_checkin
WHERE date BETWEEN 20171201 AND 20171231
AND event_name="SQL Symphony Concert"
GROUP BY person_id
HAVING COUNT (*)>=3

 * sqlite:///sql-murder-mystery.db
Done.


person_id,visits
24556,3
99716,3


In [20]:
%%sql

WITH CTE AS (
SELECT person_id,
COUNT (*) as visits
FROM facebook_event_checkin
WHERE date BETWEEN 20171201 AND 20171231
AND event_name="SQL Symphony Concert"
GROUP BY person_id
HAVING COUNT (*)>=3
)

SELECT p.*, fb.*
FROM drivers_license as dl
INNER JOIN person as p on dl.id=p.license_id
INNER JOIN CTE as fb on fb.person_id=p.id
WHERE hair_color = "red"
AND height >= 65
AND height <= 67
AND car_make LIKE "%tesla%"
AND car_model LIKE "Model S"
AND gender = "female"
LIMIT 100

 * sqlite:///sql-murder-mystery.db
Done.


id,name,license_id,address_number,address_street_name,ssn,person_id,visits
99716,Miranda Priestly,202298,1883,Golden Ave,987756388,99716,3


## Miranda Priestly is the Mastermind!

`INSERT INTO solution VALUES (1, 'Miranda Priestly');`

`SELECT value FROM solution;`